In [192]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import math

In [193]:
#parameters
scan_id=3
sample_rate=10
no_of_lines =50
line_tolarance=1.5 #threshold for line 
model_name="svm"

In [208]:
def process_data(file, sample_rate = sample_rate):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    point_array_sampled = []
    for i in range(0,len(coord),sample_rate):
        point_array_sampled.append(coord[i])
        
    point_array_sampled = np.array(point_array_sampled)
        
    return np.array( point_array_sampled)

In [209]:
def plot_graph(point_data,outline_points=None, lines=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create cubic bounding box to simulate equal aspect ratio
    X = point_data[:,0]
    Y = point_data[:,1]
    Z = point_data[:,2]

    ax.scatter(point_data[:,0], point_data[:,1], point_data[:,2], s=1)

    #bounding box to show foot in scale
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())

    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')
        
    if (outline_points is not None):
        flatten = outline_points.reshape(no_of_lines*2,3)
        ax.scatter(flatten[:,0], flatten[:,1], flatten[:,2], s=500/no_of_lines, color='red')

    plt.grid()

In [210]:
def project_points(point_array, axis=1, height_limit=100000): #axis values x=0/y=1/z=2 implement limit
    
    point_data=point_array.copy()
    
    if(axis==0):
        point_data[:,0]=0
    if(axis==1):
        point_data[:,1]=0
    if(axis==2):
        point_data[:,2]=0
        
    return point_data

In [211]:
def get_outline(point_data, no_of_lines = no_of_lines, line_tolarance = line_tolarance):
    
    z_min = min(point_data[:,2])
    z_max =max(point_data[:,2])
    
    step = (z_max-z_min)/(no_of_lines+1)
    
    lines = np.arange(z_min, z_max, step)[1:][:no_of_lines]
    
    outline_points=[]
    
    for line in lines:
        line_points=[]
        candidates = point_data[np.where(np.logical_and(point_data[:,2]>line, point_data[:,2]<line+line_tolarance))]
        
        if(len(candidates)==0 or len(outline_points)==no_of_lines):
            break
            
        line_points.append(candidates[np.where(candidates[:,0]==min(candidates[:,0]))][0])
        line_points.append(candidates[np.where(candidates[:,0]==max(candidates[:,0]))][0])
        
        
        outline_points.append(line_points)
        
    return np.array(outline_points).reshape((no_of_lines,2,3)),lines

In [212]:
def read_dataset():
    all_set = pd.read_csv("Datasets/all_data.csv", sep=',',header=0)
    train_set=pd.read_csv("Datasets/train_set.csv", sep=',',header=0)
    test_set=pd.read_csv("Datasets/test_set.csv", sep=',',header=0)
    
    return all_set.drop(all_set.index[[0,1]]),train_set.drop(train_set.index[[0,1]]), test_set.drop(test_set.index[[0,1]])

In [213]:
def load_model(model=model_name):
    if(model=="svm"):
        return pickle.load(open("Models/SVM_Classifier.pkcls", 'rb'))
    else:
        return

In [214]:
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]

In [253]:
def get_cross_sections(point_array, lines):
    cross_secs=[]
    for line in lines:
        cross_secs.append(point_array[np.where(point_array[:,2]==find_nearest(point_array[:,2],line))])
    return np.array(cross_secs)

In [254]:
points_0 = open("Datasets/Foot Models/points/left/3/sub423ll.pts")
point_array_0 = process_data(points_0)
projection_0 = project_points(point_array=point_array_0)
outline_0 =get_outline(projection_0)
outline_points_0= outline_0[0]
lines_0=outline_0[1]
cross_sec_0 = get_cross_sections(point_array_0,lines_0)

In [255]:
cross_sec_0.shape

(50, 36, 3)

In [223]:
plot_graph(projection_0, outline_points=outline_points_0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [224]:
dataset= read_dataset()
all_data= dataset[0]
train_set = dataset[1]
test_set = dataset[2]

In [225]:
model=load_model() #implement classification model
test_features=test_set[["No.","Cluster","Foot Length","Foot Breadth 1","Short Heel Girth"]]

In [259]:
c4_feet = np.array(train_set.loc[train_set["Cluster"]=='C4']["No."])

c4_point_arrays = []
c4_outline_points = [] 
c4_lines = []
c4_cross_secs=[]

for foot in c4_feet:
    points = open("Datasets/Foot Models/points/left/3/"+foot+"3ll.pts")
    point_array = process_data(points)
    projection = project_points(point_array)
    outline =get_outline(projection)
    outline_points= outline[0]
    lines=outline[1]

    c4_outline_points.append(outline_points)
    c4_point_arrays.append(point_array)
    c4_lines.append(lines)
    c4_cross_secs.append(get_cross_sections(point_array,lines))

In [53]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

# objective function to minimize
def objective(x):
    func = 0
    for i in range(0,no_of_lines):
        for j in range(0,2):
            weighted_point = 0
            for k in range(0,len(c4_outline_points)):
                weighted_point = weighted_point + x[k]*c4_outline_points[k][i][j] 
            weighted_point=weighted_point/len(c4_outline_points)
            func = func+ np.linalg.norm(weighted_point-outline_points_0[i][j])
    return func

# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded

# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
    for gen in range(n_iter):
        print("Processing Gen:",gen)
        # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
                
#                 val=0
#                 for j in range(len(decoded[i])):
#                     val=val+c4_outline_points[j]*decoded[i][j]
#                 print(val.shape)
#                 plot_graph(projection_0, outline_points=val)
                    
                
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

# define range for input
bounds = [[[0.5,1.5]]*len(c4_outline_points)][0] #negative range will not provide smooth shape. only a scaling a factor should be determined
# define the total iterations
n_iter = 1000 # take around 1 hour for 5000 iterations for cluster size of 34
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))


#set bound to (-1 to 1) and normalize the foots to max length foot

Processing Gen: 0
>0, new best f([1.191192626953125, 1.0047149658203125, 1.0037689208984375, 1.2776336669921875, 1.2327728271484375, 1.0697784423828125, 0.69171142578125, 1.0360107421875, 1.30255126953125, 1.478729248046875, 0.5605621337890625, 0.8529052734375, 0.87994384765625, 0.820465087890625, 1.33917236328125, 0.972900390625, 0.6305389404296875, 1.48309326171875, 1.094879150390625, 0.887359619140625, 0.648468017578125, 0.722869873046875, 0.5145721435546875, 1.0043792724609375, 0.74151611328125, 0.501922607421875, 1.3036651611328125, 1.4486541748046875, 1.269012451171875, 0.7297515869140625, 0.5657958984375, 1.2857818603515625, 1.14764404296875, 1.2700653076171875]) = 265.742344
>0, new best f([1.0989990234375, 0.5574188232421875, 0.9318389892578125, 1.1005859375, 0.667938232421875, 1.0846099853515625, 0.9431610107421875, 0.604400634765625, 1.310150146484375, 1.0668487548828125, 1.0860595703125, 1.215118408203125, 1.42242431640625, 1.4615325927734375, 0.8700408935546875, 0.78651428

In [57]:
decoded

[0.6251373291015625,
 0.5001220703125,
 1.2129364013671875,
 0.6605072021484375,
 1.24993896484375,
 1.0177764892578125,
 1.3748016357421875,
 0.5667724609375,
 1.499908447265625,
 0.5000457763671875,
 1.49993896484375,
 0.5328216552734375,
 0.673553466796875,
 0.996826171875,
 1.001251220703125,
 0.7527313232421875,
 0.781219482421875,
 0.5010223388671875,
 1.4999847412109375,
 1.468719482421875,
 1.499969482421875,
 0.5461273193359375,
 0.5000457763671875,
 0.7686767578125,
 0.5000457763671875,
 1.3124542236328125,
 1.4999237060546875,
 1.2966766357421875,
 1.3120574951171875,
 1.499847412109375,
 1.468719482421875,
 1.0000152587890625,
 1.1243133544921875,
 0.7499237060546875]

In [265]:
val=0
for i in range(len(decoded)):
    val=val+c4_outline_points[i]*decoded[i]
val = val/len(decoded)

In [266]:
plot_graph(projection_0, outline_points=val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [267]:
sum(decoded)

33.99481201171875

In [285]:
cross=0
for j in range(len(decoded)):
    cross=cross+c4_cross_secs[j]*decoded[j]
cross = cross/len(decoded)

In [298]:
cross=cross.reshape([(50*36),3])

In [299]:
cross.shape

(1800, 3)

In [300]:
plot_graph(cross, outline_points=val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [301]:
plot_graph(point_array_0, outline_points=val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [306]:
a_file = open("Datasets/Results/sub423ll.pts", "w")
for row in cross:
    np.savetxt(a_file, row)
